# EventBuilder Demo
  - load configuration from yaml file
  - load filter file
  - get traces
  - Instantiate EventBuilder class
      - Add OptimumFilterTrigger object for each triggers
      - Find triggers
      - Merge coincident triggers

In [1]:
import numpy as np
import vaex as vx
from detprocess import RawData, YamlConfig, FilterData, EventBuilder, OptimumFilterTrigger
from pprint import pprint
import qetpy as qp

### User input: Raw data path, yaml configuration path, and selected event

In [2]:
# continuous raw data path
raw_path = '/sdata1/runs/run47/raw/continuous_I2_D20240719_T172034'

# configuration file
yaml_file = '/data/users/serfass/run47/run47_config.html'

# selected event
series_number = 220240719172554 
event_number = 500002


### Raw data 

In [3]:
rawdata_obj = RawData(raw_path, data_type='cont', series=None)
rawdata_obj.describe()

# channels in raw data
available_channels = rawdata_obj.get_available_channels()
print(f'\nAvailable channels: {available_channels}')

# sample rate
sample_rate = rawdata_obj.get_sample_rate()

INFO: Building file map for raw data group continuous_I2_D20240719_T172034
Raw data group: continuous_I2_D20240719_T172034
Base path: /sdata1/runs/run47/raw
Number of series:
 - open continuous data: 6 series

Available channels: ['Mv3025pcBigFinsLeft', 'Mv3025pcBigFinsRight', 'Mv3025pcRegularLeft', 'Mv3025pcRegularRight']


### Read configuration file

In [4]:
yaml_obj = YamlConfig(yaml_file, available_channels)
trigger_config = yaml_obj.get_config('trigger')
pprint(trigger_config)

# filter file
filter_file = trigger_config['overall']['filter_file']


# list of individual channels that will be used for triggering
trigger_channels = trigger_config['channel_list']

{'channel_list': ['Mv3025pcBigFinsLeft', 'Mv3025pcBigFinsRight'],
 'channels': {'of2x1_shared_Melange025pcBigFins': {'channel_name': 'Mv3025pcBigFinsLeft|Mv3025pcBigFinsRight',
                                                   'csd_tag': 'default',
                                                   'pileup_window_msec': 2,
                                                   'run': True,
                                                   'template_tag': 'shared',
                                                   'threshold_sigma': 4.0},
              'of2x1_single_left_Melange025pcBigFins': {'channel_name': 'Mv3025pcBigFinsLeft|Mv3025pcBigFinsRight',
                                                        'csd_tag': 'default',
                                                        'pileup_window_msec': 2,
                                                        'run': True,
                                                        'template_tag': 'single_left',
                          

### Load filter file

In [5]:
filterdata_obj = FilterData()
filterdata_obj.load_hdf5(filter_file)
#filterdata_obj.describe()

INFO: Loading filter data from file /data/users/serfass/run47/r47_072324_filter_dm.hdf5


### Load raw data traces

In [6]:
traces, admins = rawdata_obj.get_traces(series_nums=series_number, event_nums=event_number, 
                                        channels=trigger_channels,
                                        include_metadata=True)

# convert from 3D to 2D
# the order in array follow trigger_channels
traces = traces[0,:]
trace_admin = admins[0]

### Instantiate EventBuilder and add OptimumFilterTrigger objects for each triggers

In [7]:
evtbuilder_obj = EventBuilder()

In [8]:
# loop trigger channel from yaml config
evtbuilder_obj.clear_event()
for trig_chan, trig_data in trigger_config['channels'].items():
   
    # channel name
    channel_name = trig_data['channel_name']
    
    # get template
    template_tag = trig_data['template_tag']
    template, template_time, template_metadata = (
        filterdata_obj.get_template(
            channel_name, tag=template_tag,
            return_metadata=True)
    )
    nb_pretrigger_samples = template_metadata['nb_pretrigger_samples']
    
    # get noise csd/psd
    noise_tag = 'default'
    if 'noise_tag' in trig_data:
        noise_tag = trig_data['noise_tag']
    elif 'csd_tag' in trig_data:
        noise_tag = trig_data['csd_tag']
    elif 'psd_tag' in trig_data:
        noise_tag = trig_data['psd_tag']
            
    noise, noise_freqs, noise_metadata = (
        filterdata_obj.get_csd(
            channel_name,
            tag=noise_tag,
            fold=False,
            return_metadata=True)
    )
    
    # instantiate optimal filter trigger
    oftrigger_obj = OptimumFilterTrigger(
        channel_name, sample_rate, template, noise,
        nb_pretrigger_samples,
        trigger_name=trig_chan
    )

    # add in EventBuilder
    print(f'Adding trigger channel: {trig_chan}')
    evtbuilder_obj.add_trigger_object(trig_chan, oftrigger_obj) 


Adding trigger channel: of2x1_shared_Melange025pcBigFins
Adding trigger channel: of2x1_single_left_Melange025pcBigFins
Adding trigger channel: of2x1_single_right_Melange025pcBigFins


### Acquire triggers

In [9]:
for trig_chan, trig_data in trigger_config['channels'].items():
   
    # channel name
    channel_name = trig_data['channel_name']
    channel_list = qp.utils.convert_channel_name_to_list(channel_name)
    
    # threshold
    threshold = trig_data['threshold_sigma']
    pileup_window_msec = trig_data['pileup_window_msec']

    # trace (need to be correct order)
    indices = [np.where(np.array(trigger_channels) == ch)[0][0] for ch in channel_list]
    trace = traces[indices,:]
    
    
    # acquire trigger
    evtbuilder_obj.acquire_triggers(
        trig_chan,
        trace,
        threshold,
        pileup_window_msec=pileup_window_msec
    )

### Dataframe **before** merging coincident triggers

In [10]:
df = evtbuilder_obj.get_event_df()

# check time difference
trigger_times = df['trigger_time'].values
time_differences = np.diff(trigger_times)
max_diff = np.max(time_differences)
min_diff  = np.min(time_differences)

print(f'Maximum time between adjacent triggers = {max_diff*1e3} msec')
print(f'Minimum time between adjacent triggers = {min_diff*1e3} msec\n')

df

Maximum time between adjacent triggers = 125.51519999999994 msec
Minimum time between adjacent triggers = 0.0 msec



#,trigger_delta_chi2,trigger_time,trigger_index,trigger_pileup_window,trigger_threshold_sigma,trigger_type,trigger_amplitude_0,trigger_amplitude,trigger_delta_chi2_of2x1_shared_Melange025pcBigFins,trigger_time_of2x1_shared_Melange025pcBigFins,trigger_index_of2x1_shared_Melange025pcBigFins,trigger_pileup_window_of2x1_shared_Melange025pcBigFins,trigger_threshold_sigma_of2x1_shared_Melange025pcBigFins,trigger_type_of2x1_shared_Melange025pcBigFins,trigger_amplitude_0_of2x1_shared_Melange025pcBigFins,trigger_amplitude_of2x1_shared_Melange025pcBigFins,trigger_channel,trigger_delta_chi2_of2x1_single_left_Melange025pcBigFins,trigger_time_of2x1_single_left_Melange025pcBigFins,trigger_index_of2x1_single_left_Melange025pcBigFins,trigger_pileup_window_of2x1_single_left_Melange025pcBigFins,trigger_threshold_sigma_of2x1_single_left_Melange025pcBigFins,trigger_type_of2x1_single_left_Melange025pcBigFins,trigger_amplitude_0_of2x1_single_left_Melange025pcBigFins,trigger_amplitude_of2x1_single_left_Melange025pcBigFins,trigger_delta_chi2_of2x1_single_right_Melange025pcBigFins,trigger_time_of2x1_single_right_Melange025pcBigFins,trigger_index_of2x1_single_right_Melange025pcBigFins,trigger_pileup_window_of2x1_single_right_Melange025pcBigFins,trigger_threshold_sigma_of2x1_single_right_Melange025pcBigFins,trigger_type_of2x1_single_right_Melange025pcBigFins,trigger_amplitude_0_of2x1_single_right_Melange025pcBigFins,trigger_amplitude_of2x1_single_right_Melange025pcBigFins
0,16.158275875109712,0.0209832,26229,2500,4.0,4,-0.2571636193816802,-0.2571636193816802,--,--,--,--,--,--,--,--,of2x1_single_right_Melange025pcBigFins,--,--,--,--,--,--,--,--,16.158275875109712,0.0209832,26229,2500,4.0,4,-0.2571636193816802,-0.2571636193816802
1,16.503576980034524,0.0426264,53283,2500,4.0,4,-0.2598968775151653,-0.2598968775151653,--,--,--,--,--,--,--,--,of2x1_single_right_Melange025pcBigFins,--,--,--,--,--,--,--,--,16.503576980034524,0.0426264,53283,2500,4.0,4,-0.2598968775151653,-0.2598968775151653
2,94.96402265535968,0.0470024,58753,2500,4.0,4,0.6234355619412482,0.6234355619412482,--,--,--,--,--,--,--,--,of2x1_single_right_Melange025pcBigFins,--,--,--,--,--,--,--,--,94.96402265535968,0.0470024,58753,2500,4.0,4,0.6234355619412482,0.6234355619412482
3,17.885691273387035,0.06472,80900,2500,4.0,4,-0.2705608067288591,-0.2705608067288591,--,--,--,--,--,--,--,--,of2x1_single_right_Melange025pcBigFins,--,--,--,--,--,--,--,--,17.885691273387035,0.06472,80900,2500,4.0,4,-0.2705608067288591,-0.2705608067288591
4,26.06511733272973,0.06866,85825,2500,4.0,4,0.5593210531940085,0.5593210531940085,26.06511733272973,0.06866,85825,2500,4.0,4,0.5593210531940085,0.5593210531940085,of2x1_shared_Melange025pcBigFins,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,20.204127303522412,9.9318456,12414807,2500,4.0,4,0.49243778907309493,0.49243778907309493,20.204127303522412,9.9318456,12414807,2500,4.0,4,0.49243778907309493,0.49243778907309493,of2x1_shared_Melange025pcBigFins,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--
750,25.111968562381747,9.9318488,12414811,2500,4.0,4,0.4746476967050284,0.4746476967050284,--,--,--,--,--,--,--,--,of2x1_single_left_Melange025pcBigFins,25.111968562381747,9.9318488,12414811,2500,4.0,4,0.4746476967050284,0.4746476967050284,--,--,--,--,--,--,--,--
751,25.810063315049362,9.9369864,12421233,2500,4.0,4,0.48119990530019596,0.48119990530019596,--,--,--,--,--,--,--,--,of2x1_single_left_Melange025pcBigFins,25.810063315049362,9.9369864,12421233,2500,4.0,4,0.48119990530019596,0.48119990530019596,--,--,--,--,--,--,--,--
752,22.8949170915292,9.9449872,12431234,2500,4.0,4,0.453211097712484,0.453211097712484,--,--,--,--,--,--,--,--,of2x1_single_left_Melange025pcBigFins,22.8949170915292,9.9449872,12431234,2500,4.0,4,0.453211097712484,0.453211097712484,--,--,--,--,--,--,--,--


### Merge coincident triggers

In [11]:
# get config
coincident_window_msec = trigger_config['overall']['coincident_window_msec']
print(f'Coincident window = {coincident_window_msec} msec')

# merge coincident triggers
evtbuilder_obj.build_event(
    trace_admin,
    fs=sample_rate,
    coincident_window_msec=coincident_window_msec,
)

Coincident window = 0.25 msec


### Dataframe **after** merging coincident triggers

In [12]:
df = evtbuilder_obj.get_event_df()

# check time difference
trigger_times = df['trigger_time'].values
time_differences = np.diff(trigger_times)
max_diff = np.max(time_differences)
min_diff  = np.min(time_differences)

print(f'Maximum time between adjacent triggers = {max_diff*1e3} msec')
print(f'Minimum time between adjacent triggers = {min_diff*1e3} msec\n')

df

Maximum time between adjacent triggers = 125.51519999999994 msec
Minimum time between adjacent triggers = 0.27119999999936084 msec



#,trigger_delta_chi2,trigger_time,trigger_index,trigger_pileup_window,trigger_threshold_sigma,trigger_type,trigger_amplitude_0,trigger_amplitude,trigger_delta_chi2_of2x1_shared_Melange025pcBigFins,trigger_time_of2x1_shared_Melange025pcBigFins,trigger_index_of2x1_shared_Melange025pcBigFins,trigger_pileup_window_of2x1_shared_Melange025pcBigFins,trigger_threshold_sigma_of2x1_shared_Melange025pcBigFins,trigger_type_of2x1_shared_Melange025pcBigFins,trigger_amplitude_0_of2x1_shared_Melange025pcBigFins,trigger_amplitude_of2x1_shared_Melange025pcBigFins,trigger_channel,trigger_delta_chi2_of2x1_single_left_Melange025pcBigFins,trigger_time_of2x1_single_left_Melange025pcBigFins,trigger_index_of2x1_single_left_Melange025pcBigFins,trigger_pileup_window_of2x1_single_left_Melange025pcBigFins,trigger_threshold_sigma_of2x1_single_left_Melange025pcBigFins,trigger_type_of2x1_single_left_Melange025pcBigFins,trigger_amplitude_0_of2x1_single_left_Melange025pcBigFins,trigger_amplitude_of2x1_single_left_Melange025pcBigFins,trigger_delta_chi2_of2x1_single_right_Melange025pcBigFins,trigger_time_of2x1_single_right_Melange025pcBigFins,trigger_index_of2x1_single_right_Melange025pcBigFins,trigger_pileup_window_of2x1_single_right_Melange025pcBigFins,trigger_threshold_sigma_of2x1_single_right_Melange025pcBigFins,trigger_type_of2x1_single_right_Melange025pcBigFins,trigger_amplitude_0_of2x1_single_right_Melange025pcBigFins,trigger_amplitude_of2x1_single_right_Melange025pcBigFins,processing_id,data_type,group_name,series_number,event_number,dump_number,series_start_time,group_start_time,fridge_run_start_time,fridge_run_number,event_time,trigger_prod_id
0,16.158275875109712,0.0209832,26229,2500,4.0,4,-0.2571636193816802,-0.2571636193816802,nan,nan,nan,nan,nan,nan,nan,nan,of2x1_single_right_Melange025pcBigFins,nan,nan,nan,nan,nan,nan,nan,nan,16.158275875109712,0.0209832,26229.0,2500.0,4.0,4.0,-0.2571636193816802,-0.2571636193816802,--,Test,continuous_I2_D20240719_T172034,220240719172554,500002,5,1721435616,1721435616,1721435616,47,1721435615,1
1,16.503576980034524,0.0426264,53283,2500,4.0,4,-0.2598968775151653,-0.2598968775151653,nan,nan,nan,nan,nan,nan,nan,nan,of2x1_single_right_Melange025pcBigFins,nan,nan,nan,nan,nan,nan,nan,nan,16.503576980034524,0.0426264,53283.0,2500.0,4.0,4.0,-0.2598968775151653,-0.2598968775151653,--,Test,continuous_I2_D20240719_T172034,220240719172554,500002,5,1721435616,1721435616,1721435616,47,1721435615,2
2,94.96402265535968,0.0470024,58753,2500,4.0,4,0.6234355619412482,0.6234355619412482,nan,nan,nan,nan,nan,nan,nan,nan,of2x1_single_right_Melange025pcBigFins,nan,nan,nan,nan,nan,nan,nan,nan,94.96402265535968,0.0470024,58753.0,2500.0,4.0,4.0,0.6234355619412482,0.6234355619412482,--,Test,continuous_I2_D20240719_T172034,220240719172554,500002,5,1721435616,1721435616,1721435616,47,1721435615,3
3,17.885691273387035,0.06472,80900,2500,4.0,4,-0.2705608067288591,-0.2705608067288591,nan,nan,nan,nan,nan,nan,nan,nan,of2x1_single_right_Melange025pcBigFins,nan,nan,nan,nan,nan,nan,nan,nan,17.885691273387035,0.06472,80900.0,2500.0,4.0,4.0,-0.2705608067288591,-0.2705608067288591,--,Test,continuous_I2_D20240719_T172034,220240719172554,500002,5,1721435616,1721435616,1721435616,47,1721435615,4
4,26.06511733272973,0.06866,85825,2500,4.0,4,0.5593210531940085,0.5593210531940085,26.06511733272973,0.06866,85825.0,2500.0,4.0,4.0,0.5593210531940085,0.5593210531940085,of2x1_shared_Melange025pcBigFins,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,--,Test,continuous_I2_D20240719_T172034,220240719172554,500002,5,1721435616,1721435616,1721435616,47,1721435615,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,18.403982299960305,9.9128344,12391043,2500,4.0,4,-0.2744529628774232,-0.2744529628774232,nan,nan,nan,nan,nan,nan,nan,nan,of2x1_single_right_Melange025pcBigFins,nan,nan,nan,nan,nan,nan,nan,nan,18.403982299960